Installing & Importing Neccessary Packages

In [ ]:
#Installing necessary packages

!pip install tableau-api-lib

In [23]:
#Importing necesssary packages
import pandas as pd
import os
import urllib.parse
import time
import urllib
from tableau_api_lib import TableauServerConnection as TSC
from tableau_api_lib.utils import querying,flatten_dict_column

Setting up the configuration to connect to tableau server 

In [25]:

#setting up the Configuration which we will use for connecting python with tableau server or tableau cloud
config={

    'SP':{                        #"SP" is name of the environment , you can give any name to it
    'server': "https:///********", #name of your server where your dashboard is published
    'api_version': '3.##' ,
    'personal_access_token_name': "P#####", # name of the personal token which you can create by going to your account settings
    'personal_access_token_secret': "##############", # Secret key of the personal token which you can create by going to your account settings
    'site_name': "########", #site name where you are publishing your dashboard
    'site_url': "###########"}  #site url where you are publishing your dashboard

}

In [ ]:
conn=TSC (config,env='SP')
conn.sign_in()   # signing in the server using credentials 
# if you get "<Response [200]>" then you are good to go

Creating dataframes to get the necessaary information about the dashboard from which we are going to extract pdfs like workbook folder ,dashboard name , view id etc...

In [ ]:
views_df=querying.get_views_dataframe (conn) #creating dataframe having column fields like workbook, owner, project, location, id etc...

In [ ]:
views_df.head() #having a quick look at the dataset

In [ ]:
views_wb_info=flatten_dict_column(
    df=views_df
    keys=["name","id"]
    col_name='workbook'
) #flattening the necesssary information from workbook column as it contains information in the form of dictionary

In [ ]:
views_wb_info.head()  #having  a quick look at the dataset

In [ ]:
views_wb_and_project_info=flatten_dict_column(
    df=views_wb_info
    keys=["name","id"]
    col_name='project'
) #flattening the necesssary information from project column as it contains information in the form of dictionary

In [ ]:
views_wb_and_project_info.head() #having a quick look at the dataset

In [ ]:
views_wb_and_project_info=views_wb_and_project_info[views_wb_and_project_info['project name']=='######']# name of your project folder where you have kept your dashboard

In [ ]:
views_wb_and_project_info=views_wb_and_project_info[views_wb_and_project_info['workbook name']=='######'] # name of your tableau workbook 

In [ ]:
df=pd.read_excel("###########') #reading the excel where our values of our filters are present

In [ ]:
eco_value=df['Region'].to_list() #extracting the necessary column having filter values from excel and converting it to list so that we can loop over it

In [ ]:
eco_code=df['Code'].to_list() #extracting the necessary column having filter codes from excel and converting it to list so that we can loop over it

In [37]:
ecosystem_fil=urllib.parse.quote('Region') #name of the exact filter which you are using in your dashboard
view_id="##############" # view_id means id of your dashboards in the workbook
folder="######" #name of the folder where you want to store your pdfs output
os.makedirs(folder,exist_ok=True) #creation of folder if it doesn't exists

for i, j in zip(eco_value,eco_code):   #mapping each value of eco_value with eco_code
    eco_list=[i,'####'] #customising it as per the need of the filter values which we needed
    #encode each value individually and join it with %2C
    ecosystem_fil_value="%2C".join(urllib.parse.quote(e.strip()) for e in eco_list)
    custom_url_para={                                            #customising the url_para as per the need , this is going to filter the values in the workbook
    "Ecosystem":f"vf_{ecosystem_fil}={ecosystem_fil_value}",}
    print(f"request params for{j} : {custom_url_para}")

    view_pdf_response=conn.query_view_pdf(view_id=view_id,parameter_dict=custom_url_para)  ##getting the pdf response


    #It will be useful if the pdfs are not getting printed, error checking code
    if view_pdf_response_code !=200:
        print(f"Error: Received Status code {view_pdf_response.status_code} for {j}")
        continue
    
    if b '%PDF' not in view_pdf_response_content [:10]:
        print(f"Warning : Response for {j} does not appear to be a PDF!")
        print (view_pdf_response.content[:200]) #Debug output
        continue

    file_path=os.path.join(folder,f"{j}.pdf")  #here we are creating the file path for each value of the filter
    with open (file_path,"wb") as f:
        f.write(view_pdf_response.content)  # here we are writing content of the response of the pdf in the mentioned filepath
    print(f"Saved Pdf for {j} at {file_path} (os.path.getsize(file_path)} bytes)") #if succesfully print , we will get this in the output
    time.sleep(1) #giving pause of 1 second 

print("Done") #successfully completed
      

SyntaxError: incomplete input (592535059.py, line 10)